In [8]:
from sage.misc.misc_c import prod

from lib.Hypermatrix_Algebra_tst import HM, var_list
from sage.symbolic.operators import add_vararg

from itertools import permutations, combinations, chain
from random import randrange
from datetime import datetime

import gc

# import numpy

In [9]:
sz = 3; HV = HM(*(sz for i in range(sz)), 'v')

In [10]:
HV.printHM() # only valid for sz < 4 # fixed!
# print(V)

[:, :, 0]=
[v000 v010 v020]
[v100 v110 v120]
[v200 v210 v220]

[:, :, 1]=
[v001 v011 v021]
[v101 v111 v121]
[v201 v211 v221]

[:, :, 2]=
[v002 v012 v022]
[v102 v112 v122]
[v202 v212 v222]



In [11]:
def lexSymbol(digits, sz, terms, f):
    raw = int(str(f)[1:]) # will only work when sz <= 10 # also fixed!
    return sum((raw // 10**(digits*(terms-i-1))) % 10**digits * sz**i for i in range(terms))


lexSymbol(1, sz, sz, HV[tuple(sz-1 for i in range(sz))])

26

In [12]:
E = HM(*(sz for i in range(2)), 'a')
E.printHM()

[:, :]=
[a00 a01 a02]
[a10 a11 a12]
[a20 a21 a22]


In [13]:


def listFromSymbol(digits, terms, f):
    raw = int(str(f)[1:])
    return [(raw // 10**(digits*(terms-i-1))) % 10**digits for i in range(terms)]



def composeSymbols(HV, digits, terms, f1, f2):
    
    f1list = listFromSymbol(digits, terms, f1)
    f2list = listFromSymbol(digits, terms, f2)
    
    return HV[tuple(f1list[f2list[i]] for i in range(sz))]

# randFunc = lambda: (randrange(sz) for i in range(sz))
def genRandFunc(sz):
    history = []
    def randFunc(ctx):
        nonlocal history
        func = []
        for i in range(sz):
            func.append(randrange(sz))
            yield func[-1]
        history.append(func + [ctx])
        
    return randFunc, lambda: history
randFunc, getHistory = genRandFunc(sz)

print(composeSymbols(HV, 1, sz, HV[tuple(randFunc('cmpSym'))], HV[tuple(randFunc('cmpSym'))]))
# print(composeSymbols(HV, 1, sz, HV[1,1,0], HV[0,1,0]))
# print(composeSymbols(HV, 1, sz, HV[0,1,0], HV[1,1,0]))

def lexList(sz, f):
    return sum((d * sz**i for i, d in enumerate(f)))

def composeLists(terms, f1, f2):
    return [f1[f2[i]] for i in range(terms)]

print(lexList(sz, composeLists(sz, list(randFunc('lexLst')), list(randFunc('lexLst')))))

def listFromLex(sz, terms, lex):
    return [(lex // (sz ** i)) % sz for i in range(terms)]

def SymbolFromLex(HV, sz, terms, lex):
    return HV[tuple((lex // (sz ** i)) % sz for i in range(terms))]

def composeLex(sz, terms, f1, f2):
    f1list = listFromLex(sz, terms, f1)
    f2list = listFromLex(sz, terms, f2)
    
    return lexList(sz, (f1list[f2list[i]] for i in range(sz)))

genHMLex = lambda f1: (lambda f2: (lambda digits: str(composeLex(sz, sz, f1, f2)).zfill(digits)))

print(genHMLex(lexList(sz, randFunc('HMLex')))(lexList(sz, randFunc('HMLex')))(2))

print(SymbolFromLex(HV, sz, sz, lexList(sz, composeLists(sz, list(randFunc('symLex')), list(randFunc('symLex'))))))

def SymbolFromList(f):
    return HV[tuple(f)]

print(SymbolFromList(list(randFunc('symLst'))))
# print(SymbolFromList([1,1,1]))

def get_ident_func(sz):
    ident = [i for i in range(sz)]
    return ident, lexList(sz, ident)

print(getHistory())

v121
12
18
v121
v100
[[2, 1, 2, 'cmpSym'], [1, 0, 1, 'cmpSym'], [2, 1, 0, 'lexLst'], [2, 1, 1, 'lexLst'], [1, 0, 2, 'HMLex'], [1, 1, 2, 'HMLex'], [2, 1, 1, 'symLex'], [2, 0, 1, 'symLex'], [1, 0, 0, 'symLst']]


In [14]:
HE = HM(*(sz**sz for i in range(2)), 'e', func=genHMLex)

HE.printHM()

[:, :]=
[e000000 e000100 e000200 e000300 e000400 e000500 e000600 e000700 e000800 e000900 e001000 e001100 e001200 e001300 e001400 e001500 e001600 e001700 e001800 e001900 e002000 e002100 e002200 e002300 e002400 e002500 e002600]
[e010013 e010112 e010212 e010310 e010409 e010509 e010610 e010709 e010809 e010904 e011003 e011103 e011201 e011300 e011400 e011501 e011600 e011700 e011804 e011903 e012003 e012101 e012200 e012300 e012401 e012500 e012600]
[e020026 e020124 e020224 e020320 e020418 e020518 e020620 e020718 e020818 e020908 e021006 e021106 e021202 e021300 e021400 e021502 e021600 e021700 e021808 e021906 e022006 e022102 e022200 e022300 e022402 e022500 e022600]
[e030000 e030101 e030200 e030303 e030404 e030503 e030600 e030701 e030800 e030909 e031010 e031109 e031212 e031313 e031412 e031509 e031610 e031709 e031800 e031901 e032000 e032103 e032204 e032303 e032400 e032501 e032600]
[e040013 e040113 e040212 e040313 e040413 e040512 e040610 e040710 e040809 e040913 e041013 e041112 e041213 e041313 e041412

In [9]:
# monoids = prod(1 + HE[x, y, composeLex(sz, sz, x, y)] * Lx[x] * Ly[y] * Lz[composeLex(sz, sz, x, y)] for x in range(sz**sz) for y in range(sz**sz))

_, ident = get_ident_func(sz); rng_no_ident = lambda: (i for i in range(sz**sz) if i != ident)

od = 3; Lx = var_list('x', sz**sz); Ly = var_list('y', sz**sz); Lz = var_list('z', sz**sz * od**2);
monoids = HE[ident, ident] * Lz[ident] * sum(prod(HE[x, y] * Lx[x] * Ly[y] * Lz[composeLex(sz, sz, x, y) + (i*(od-1) + j) * sz**sz] for i,x in enumerate(f) for j,y in enumerate(f)) for f in combinations(rng_no_ident(), od - 1))



# print(monoids.expand().diff(Lx[ident]).diff(Ly[ident]).diff(Lz[ident]))

# expand = monoids.expand()

# print(monoids)

In [10]:
with_ident = monoids.diff(Lz[ident])

#print(with_ident)

In [11]:
def flatten_num(itr, layers):
    for i in range(layers):
        itr = chain.from_iterable(itr)
    return itr

# print(HV)
# print(list(flatten_num(HV.listHM(), sz-1)))

In [12]:
# temp = 1
# for i in range(sz**sz):
#      temp2 = with_ident.diff(Lx[i]).diff(Ly[i])
#      temp += temp2.diff(Lz[ident]) + temp2.diff(Lz[i])

In [14]:
# eval_temp = temp({Lx[0]:0,Lx[1]:0,Lx[2]:0,Lx[3]:0,Ly[0]:0,Ly[1]:0,Ly[2]:0,Ly[3]:0,Lz[0]:0,Lz[1]:0,Lz[2]:0,Lz[3]:0})
# eval_temp = temp(dict.fromkeys((v for vlist in [Lx, Ly, Lz] for v in vlist), 0))

# zval_xy = dict.fromkeys((v for vlist in [Lx, Ly] for v in vlist), 0)

cuml = 1
zval_xy = dict.fromkeys((v for v in chain(Lx, Ly)), 0)
zval_z  = dict.fromkeys((v for v in Lz),            0)

# For monoid of size 2
# for i in rng_no_ident():
#     temp = (with_ident.diff(Lx[i]).diff(Ly[i]))(zval_xy)
#     cuml += (temp.diff(Lz[ident]))(zval_z) + (temp.diff(Lz[i]))(zval_z)

# For monoid of size 3
for k, f in enumerate(combinations(rng_no_ident(), od - 1)):
    # print(f)
    i, j = f
    temp = (with_ident.diff(Lx[i]).diff(Ly[i]).diff(Lx[i]).diff(Ly[j]).diff(Lx[j]).diff(Ly[i]).diff(Lx[j]).diff(Ly[j]))(zval_xy)
    
    temp1 = temp.diff(Lz[i]) #aaa
    cuml += (temp.diff(Lz[j]).diff(Lz[ident+sz**sz]).diff(Lz[ident+2 * sz**sz]).diff(Lz[i+3 * sz**sz]))(zval_z)
    
    temp2 = temp1.diff(Lz[i+sz**sz]) #aba
    temp3 = temp1.diff(Lz[j+sz**sz]) #abb
    
    temp1 = temp2.diff(Lz[i+2 * sz**sz]) #baa
    cuml += (temp2.diff(Lz[j+2 * sz**sz]).diff(Lz[j+3 * sz**sz]))(zval_z)
    
    cuml += (temp1.diff(Lz[ident+3 * sz**sz]))(zval_z) + (temp1.diff(Lz[i+3 * sz**sz]))(zval_z) + (temp1.diff(Lz[j+3 * sz**sz]))(zval_z)
    
    cuml += (temp3.diff(Lz[j+2 * sz**sz]).diff(Lz[i+3 * sz**sz]))(zval_z) + (temp3.diff(Lz[i+2 * sz**sz]).diff(Lz[j+3 * sz**sz]))(zval_z)
    
    temp2 = temp.diff(Lz[j+3 * sz**sz]) #bbb
    
    cuml += (temp2.diff(Lz[i+2 * sz**sz]).diff(Lz[i+sz**sz]).diff(Lz[j]))(zval_z)
    temp3 = temp2.diff(Lz[j+2 * sz**sz]).diff(Lz[j+sz**sz]) #abb
    
    cuml += (temp3.diff(Lz[ident]))(zval_z) + (temp3.diff(Lz[i]))(zval_z) + (temp3.diff(Lz[j]))(zval_z)
    
    if k % 40 == 0:
        print(i,j,k,datetime.now())
    if k % 20 == 19: # 120 == 119:
        if k % 120 == 119: # 480 == 479:
            gc.collect(generation=2)
        else:
            gc.collect(generation=1)
    

cuml = (cuml - 1).expand()
print(cuml)

final_symb = sum(prod(e for e in term.arguments()) for term in cuml.operands()) if cuml.operator() == add_vararg else prod(e for e in cuml.arguments())
final_count = final_symb(dict.fromkeys((e for term in final_symb.operands() for e in term.arguments()), 1))

0 1 0 2021-05-02 05:36:55.660875
0 41 40 2021-05-02 05:37:33.518120
0 81 80 2021-05-02 05:38:11.459110
0 121 120 2021-05-02 05:38:51.361126
0 161 160 2021-05-02 05:39:27.199243
0 201 200 2021-05-02 05:40:04.127372
0 242 240 2021-05-02 05:40:44.306111


KeyboardInterrupt: 

In [14]:
print(final_symb)
print(final_count)

e000000*e000300*e030000*e030303*e212121 + e000000*e000600*e060000*e060600*e212121 + e000000*e000900*e090000*e090900*e212121 + e030303*e031010*e100310*e101003*e212121 + e070711*e071121*e110721*e111107*e212121 + e000000*e001200*e120000*e121212*e212121 + e010112*e011201*e120101*e121212*e212121 + e030303*e031212*e120303*e121212*e212121 + e000000*e001300*e130013*e131313*e212121 + e030303*e031313*e130313*e131313*e212121 + e040413*e041313*e130413*e131313*e212121 + e050521*e051313*e130513*e131313*e212121 + e121212*e121313*e131213*e131313*e212121 + e131313*e131413*e141313*e141413*e212121 + e000000*e001500*e150000*e151521*e212121 + e000000*e001800*e180000*e181818*e212121 + e080818*e081808*e180808*e181818*e212121 + e131313*e132213*e212121*e221313*e222222 + e171722*e172217*e212121*e221717*e222222 + e181818*e182218*e212121*e221822*e222222 + e030303*e032303*e212121*e230323*e232323 + e131313*e132313*e212121*e231313*e232323 + e161623*e162316*e212121*e231616*e232323 + e212121*e222222*e222323*e232222*e2

In [16]:
HE = HM(*(sz**sz for i in range(3)), 'e'); HE.printHM()

[:, :, 0]=
[e000000 e000100 e000200 e000300 e000400 e000500 e000600 e000700 e000800 e000900 e001000 e001100 e001200 e001300 e001400 e001500 e001600 e001700 e001800 e001900 e002000 e002100 e002200 e002300 e002400 e002500 e002600]
[e010000 e010100 e010200 e010300 e010400 e010500 e010600 e010700 e010800 e010900 e011000 e011100 e011200 e011300 e011400 e011500 e011600 e011700 e011800 e011900 e012000 e012100 e012200 e012300 e012400 e012500 e012600]
[e020000 e020100 e020200 e020300 e020400 e020500 e020600 e020700 e020800 e020900 e021000 e021100 e021200 e021300 e021400 e021500 e021600 e021700 e021800 e021900 e022000 e022100 e022200 e022300 e022400 e022500 e022600]
[e030000 e030100 e030200 e030300 e030400 e030500 e030600 e030700 e030800 e030900 e031000 e031100 e031200 e031300 e031400 e031500 e031600 e031700 e031800 e031900 e032000 e032100 e032200 e032300 e032400 e032500 e032600]
[e040000 e040100 e040200 e040300 e040400 e040500 e040600 e040700 e040800 e040900 e041000 e041100 e041200 e041300 e041